In [4]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.transforms import transforms

from src.model.model import MyModel
import torch


weights_path = "../netG_model_epoch_1.pth"
model = torch.load(weights_path)

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL newmodel.model.MyBigModel was not an allowed global by default. Please use `torch.serialization.add_safe_globals([MyBigModel])` or the `torch.serialization.safe_globals([MyBigModel])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with 

In [16]:
layer_names = []
def get_layer_names(model):
    for name, module in model.named_modules():
        layer_names.append(name)
    return layer_names

get_layer_names(model)

['',
 'input_proj',
 'input_proj.proj',
 'input_proj.proj.0',
 'input_proj.proj.1',
 'pos_drop',
 'encoder_0',
 'encoder_0.drop_path',
 'encoder_0.norm1',
 'encoder_0.mlp',
 'encoder_0.mlp.linear1',
 'encoder_0.mlp.linear1.0',
 'encoder_0.mlp.linear1.1',
 'encoder_0.mlp.dwconv',
 'encoder_0.mlp.dwconv.0',
 'encoder_0.mlp.dwconv.1',
 'encoder_0.mlp.linear2',
 'encoder_0.mlp.linear2.0',
 'encoder_0.mlp.eca',
 'encoder_0.norm2',
 'encoder_0.freq_mlp',
 'encoder_0.freq_mlp.linear1',
 'encoder_0.freq_mlp.linear1.0',
 'encoder_0.freq_mlp.linear1.1',
 'encoder_0.freq_mlp.dwconv',
 'encoder_0.freq_mlp.dwconv.0',
 'encoder_0.freq_mlp.dwconv.1',
 'encoder_0.freq_mlp.linear2',
 'encoder_0.freq_mlp.linear2.0',
 'encoder_0.freq_mlp.eca',
 'encoder_0.drop_path2',
 'downsample_0',
 'downsample_0.conv',
 'encoder_1',
 'encoder_1.drop_path',
 'encoder_1.norm1',
 'encoder_1.mlp',
 'encoder_1.mlp.linear1',
 'encoder_1.mlp.linear1.0',
 'encoder_1.mlp.linear1.1',
 'encoder_1.mlp.dwconv',
 'encoder_1.mlp.dw

In [22]:
from pytorch_grad_cam import GradCAM

GradCAM(model=model,target_layers=model.output_proj.proj)

In [2]:
class ImageEnhancementTarget:
    def __init__(self):
        pass

    def __call__(self, model_output):
        return (model_output).sum()

In [42]:
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch
import numpy as np
inp_image_path = "../data/uw_data/uw_data/manipulated/1imagea/6_img_.png"

image = Image.open(inp_image_path)

transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

inp = transform(image)

inp = inp.unsqueeze_(0)
print(inp.shape)
model.eval()
out = model(inp)
o = out.detach().numpy()

o = o.squeeze()
o = o.transpose(1, 2, 0)
o = np.clip(o, 0, 1)
o = o*255
o = o.astype('uint8')
print(o.shape)
Image.fromarray(o).convert('RGB').show()
targets = [ImageEnhancementTarget()]
print(targets)
cam = GradCAM(model=model,target_layers=model.output_proj.proj)

gray_cam = cam(input_tensor=inp, targets=targets)[0, :]
rgb_image = inp[0].detach().numpy()
rgb_image = rgb_image.transpose((1, 2, 0))

cam_image = show_cam_on_image(rgb_image,gray_cam,use_rgb=True)
print(cam_image.shape)
Image.fromarray(cam_image).convert("RGB").show()

torch.Size([1, 3, 256, 256])
(256, 256, 3)
(256, 256, 3)
